# Import

## Modules

In [108]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.optimize as sco
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import itertools  
import re
from datetime import datetime, timedelta

In [2]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.palettes import Dark2_5 as palette

## Data

In [3]:
portfolio = ["WES.AX", 'IVV.AX', 'QBE.AX']

metrics = ["High", "Low", "Open", "Close", "Adj Close", "Volume"]
columns = pd.MultiIndex.from_product([portfolio, metrics])

n_securities = len(portfolio)

source = 'yahoo'
start = '2017-01-01'
end = '{0}-{1}-{2}'.format(datetime.now().year, datetime.now().month, datetime.now().day)

stocks_df = pd.DataFrame(columns=columns, index=pd.date_range(start=start, end=end, freq="D"))

data = pd.DataFrame()
stocks_dict = {}

#Retrieve stock price data and save just the dividend adjusted closing prices

for share in portfolio:
        data[share] = pdr.DataReader(share, data_source=source, start=start, end=end)['Adj Close']
        temp = pdr.DataReader(share, data_source=source, start=start, end=end)[metrics]
        stocks_df[share] = temp
        
stocks_df.dropna(how="all", inplace=True)

# Preprocessing

In [134]:
data = stocks_df.loc[:, pd.MultiIndex.from_product([portfolio, ["Close"]])]

In [135]:
data.reset_index().describe()

,WES.AX,IVV.AX,QBE.AX
,Close,Close,Close
count,560.000000,554.000000,560.000000
mean,31.582808,349.666498,11.143125
std,2.203565,31.120026,1.074493
min,28.509600,298.660004,9.300000
25%,29.747900,320.139999,10.310000
50%,31.206900,350.335007,10.935000
75%,32.711375,375.132507,12.062500
max,37.384201,414.950012,13.580000


In [40]:
def build_metrics(data, portfolio, metric):

    default_days = [15, 30, 90, 180, 365, 730]
    
    for share in portfolio:
        data.loc[:, (share, "Pct Change")] = data.loc[:, (share, metric)].pct_change()
        data.loc[:, (share, "Cum Sum Pct Change")] = data.loc[:, (share, "Pct Change")].cumsum()
        for days in default_days:
            data.loc[:, (share, "Rolling mean " + str(days))] = data.loc[:, (share, "Cum Sum Pct Change")].rolling(days).mean()

    data.sort_index(axis=1, inplace=True)
    return data

In [142]:
# Todo: Plot this!

def get_trade_points(data, portfolio, short_term_metric='Rolling mean 15', long_term_metric='Rolling mean 90'):
    
    for share in portfolio:
        mask = np.argwhere(np.nan_to_num(np.diff(np.sign(data.loc[:, (share, short_term_metric)].values - data.loc[:, (share, long_term_metric)].values)))).flatten()
        indices = data.index[mask]
        
        for date in indices:
            mask_date = data.index == date
            previous_day = date - timedelta(days=1)
            mask_pd = data.index == previous_day
            
            next_day = date + timedelta(days=1)
            mask_nd = data.index == next_day
            
            variation = data.loc[mask_nd, (share, short_term_metric)].values - data.loc[mask_pd, (share, short_term_metric)].values
            if variation < 0:
                data.loc[mask_date, (share, "Trade")] = "Sell"
                
            else:
                data.loc[mask_date, (share, "Trade")] = "Buy"

        data.loc[:, (share, "Trade")].fillna("Hold", inplace=True)
    data.sort_index(axis=1, inplace=True)
    
    return indices, data

In [136]:
data = build_metrics(data, portfolio, "Close")
data.describe()

IVV.AX                                                 \
            Close Cum Sum Pct Change  Pct Change Rolling mean 15   
count  554.000000         560.000000  560.000000      546.000000   
mean   349.666498           0.117706    0.000490        0.117444   
std     31.120026           0.093935    0.008236        0.091705   
min    298.660004          -0.043685   -0.057610       -0.033392   
25%    320.139999           0.027190   -0.003343        0.031005   
50%    350.335007           0.122500    0.000843        0.120025   
75%    375.132507           0.197140    0.005268        0.194758   
max    414.950012           0.297482    0.040420        0.282552   

                                                                          \
      Rolling mean 180 Rolling mean 30 Rolling mean 365 Rolling mean 730   
count       381.000000      531.000000       196.000000              0.0   
mean          0.120390        0.117579         0.120365              NaN   
std           0.070124        0.089128         0.032293              NaN   
min           0.008899       -0.026647         0.061723              NaN   
25%           0.058450        0.029476         0.091979              NaN   
50%           0.120294        0.119204         0.124943              NaN   
75%           0.193071        0.193743         0.145982              NaN   
max           0.225328        0.279416         0.174735              NaN   

                           QBE.AX  ...                      WES.AX  \
      Rolling mean 90       Close  ... Rolling mean 90       Close   
count      471.000000  560.000000  ...      471.000000  560.000000   
mean         0.120197   11.143125  ...       -0.109741   31.582808   
std          0.082060    1.074493  ...        0.071553    2.203565   
min         -0.002119    9.300000  ...       -0.214739   28.509600   
25%          0.025581   10.310000  ...       -0.168316   29.747900   
50%          0.119466   10.935000  ...       -0.123203   31.206900   
75%          0.201833   12.062500  ...       -0.082895   32.711375   
max          0.245200   13.580000  ...        0.029755   37.384201   

                                                                       \
      Cum Sum Pct Change  Pct Change Rolling mean 15 Rolling mean 180   
count         560.000000  560.000000      546.000000       381.000000   
mean            0.048791    0.000298        0.048184         0.045495   
std             0.073808    0.010552        0.071805         0.053566   
min            -0.059080   -0.071550       -0.045346        -0.009366   
25%            -0.012790   -0.004517       -0.010850        -0.005476   
50%             0.035620    0.001053        0.034457         0.022248   
75%             0.099009    0.005462        0.099359         0.101948   
max             0.225068    0.068765        0.207193         0.130809   

                                                                         
      Rolling mean 30 Rolling mean 365 Rolling mean 730 Rolling mean 90  
count      531.000000       196.000000              0.0      471.000000  
mean         0.048086         0.044586              NaN        0.047712  
std          0.069905         0.020671              NaN        0.063974  
min         -0.043089         0.004119              NaN       -0.031312  
25%         -0.012102         0.028547              NaN        0.004949  
50%          0.031878         0.046296              NaN        0.014502  
75%          0.095540         0.061358              NaN        0.097703  
max          0.199771         0.080286              NaN        0.172186  

[8 rows x 27 columns]

In [141]:
indices, data = get_trade_points(data, portfolio)
data.describe()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  app.launch_new_instance()


IVV.AX                                                 \
            Close Cum Sum Pct Change  Pct Change Rolling mean 15   
count  554.000000         560.000000  560.000000      546.000000   
mean   349.666498           0.117706    0.000490        0.117444   
std     31.120026           0.093935    0.008236        0.091705   
min    298.660004          -0.043685   -0.057610       -0.033392   
25%    320.139999           0.027190   -0.003343        0.031005   
50%    350.335007           0.122500    0.000843        0.120025   
75%    375.132507           0.197140    0.005268        0.194758   
max    414.950012           0.297482    0.040420        0.282552   

                                                                          \
      Rolling mean 180 Rolling mean 30 Rolling mean 365 Rolling mean 730   
count       381.000000      531.000000       196.000000              0.0   
mean          0.120390        0.117579         0.120365              NaN   
std           0.070124        0.089128         0.032293              NaN   
min           0.008899       -0.026647         0.061723              NaN   
25%           0.058450        0.029476         0.091979              NaN   
50%           0.120294        0.119204         0.124943              NaN   
75%           0.193071        0.193743         0.145982              NaN   
max           0.225328        0.279416         0.174735              NaN   

                           QBE.AX  ...                      WES.AX  \
      Rolling mean 90       Close  ... Rolling mean 90       Close   
count      471.000000  560.000000  ...      471.000000  560.000000   
mean         0.120197   11.143125  ...       -0.109741   31.582808   
std          0.082060    1.074493  ...        0.071553    2.203565   
min         -0.002119    9.300000  ...       -0.214739   28.509600   
25%          0.025581   10.310000  ...       -0.168316   29.747900   
50%          0.119466   10.935000  ...       -0.123203   31.206900   
75%          0.201833   12.062500  ...       -0.082895   32.711375   
max          0.245200   13.580000  ...        0.029755   37.384201   

                                                                       \
      Cum Sum Pct Change  Pct Change Rolling mean 15 Rolling mean 180   
count         560.000000  560.000000      546.000000       381.000000   
mean            0.048791    0.000298        0.048184         0.045495   
std             0.073808    0.010552        0.071805         0.053566   
min            -0.059080   -0.071550       -0.045346        -0.009366   
25%            -0.012790   -0.004517       -0.010850        -0.005476   
50%             0.035620    0.001053        0.034457         0.022248   
75%             0.099009    0.005462        0.099359         0.101948   
max             0.225068    0.068765        0.207193         0.130809   

                                                                         
      Rolling mean 30 Rolling mean 365 Rolling mean 730 Rolling mean 90  
count      531.000000       196.000000              0.0      471.000000  
mean         0.048086         0.044586              NaN        0.047712  
std          0.069905         0.020671              NaN        0.063974  
min         -0.043089         0.004119              NaN       -0.031312  
25%         -0.012102         0.028547              NaN        0.004949  
50%          0.031878         0.046296              NaN        0.014502  
75%          0.095540         0.061358              NaN        0.097703  
max          0.199771         0.080286              NaN        0.172186  

[8 rows x 27 columns]

In [103]:
linewidth = 2
share = "WES.AX"
metric = "Cum Sum Pct Change"
# create a color iterator
colors = itertools.cycle(palette) 
rolling_columns = [days for days in data[share].columns.values.squeeze() if re.search("Rolling mean \d+", days)]
rolling_columns = ['Rolling mean 15', 'Rolling mean 90']

p = figure(plot_width = 900, plot_height = 600, title = 'Cumulative Variation', 
           x_axis_label = 'Date [Days]', x_axis_type='datetime', y_axis_label = 'Cum Sum Pct Change')

p.outline_line_color = None

p.xgrid.visible = False
p.ygrid.visible = False

p.xgrid.visible = False
p.ygrid.visible = False

p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

p.xaxis.axis_line_width = 0
p.xaxis.axis_line_color = None

p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

p.yaxis.axis_line_width = 0
p.yaxis.axis_line_color = None

p.line(data.loc[:, (share, metric)].index, data.loc[:, (share, metric)].values, line_width=linewidth, color="blue", legend=share, alpha=0.75, line_dash="dashed")
for color, rol_col in zip(colors, rolling_columns):
    p.line(data.loc[:, (share, metric)].index, data.loc[:, (share, rol_col)].values, line_width=linewidth, color=color, legend=rol_col, alpha=0.75)

p.circle(indices, data.loc[indices, (share, metric)].values, size=8, color="red", alpha=1, fill_color="white", legend="Trade")    
    
# To show in notebook
output_notebook()

# Show the plot
show(p)

Loading BokehJS ...

In [46]:
linewidth = 2
pct_change = figure(plot_width = 900, plot_height = 600, title = 'Price Plot', 
               x_axis_label = 'Date [Days]', x_axis_type='datetime', y_axis_label = 'Cumulative Percent Change [%]')
metric = "Cum Sum Pct Change"

colors = itertools.cycle(palette) 

for share, color in zip(list(set([item[0] for item in data.columns.values.squeeze()])), colors):

    # create a color iterator    
    pct_change.outline_line_color = None

    pct_change.xgrid.visible = False
    pct_change.ygrid.visible = False

    pct_change.xgrid.visible = False
    pct_change.ygrid.visible = False

    pct_change.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    pct_change.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    pct_change.xaxis.axis_line_width = 0
    pct_change.xaxis.axis_line_color = None

    pct_change.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    pct_change.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

    pct_change.yaxis.axis_line_width = 0
    pct_change.yaxis.axis_line_color = None

    pct_change.line(data.loc[:, (share, metric)].index, data.loc[:, (share, metric)].values, line_width=linewidth, color=color, legend=share, alpha=0.5)
# To show in notebook
output_notebook()

# Show the plot
show(pct_change)

Loading BokehJS ...